# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 11:53AM,246524,18,Stone-8530017,Acute Outpatient Solutions,Executing
1,Sep 15 2022 11:52AM,246523,10,8529987.1,Yusen – 3D Matrix,Released
2,Sep 15 2022 11:43AM,246522,10,0085983039,ISDIN Corporation,Released
3,Sep 15 2022 11:43AM,246522,10,0085983040,ISDIN Corporation,Released
4,Sep 15 2022 11:43AM,246522,10,0085983042,ISDIN Corporation,Released
5,Sep 15 2022 11:43AM,246522,10,0085983041,ISDIN Corporation,Released
6,Sep 15 2022 11:43AM,246522,10,0085983070,ISDIN Corporation,Released
7,Sep 15 2022 11:43AM,246522,10,0085983071,ISDIN Corporation,Released
8,Sep 15 2022 11:43AM,246522,10,0085983096,ISDIN Corporation,Released
9,Sep 15 2022 11:43AM,246522,10,0085983095,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,246519,Released,1
43,246521,Released,24
44,246522,Released,18
45,246523,Released,1
46,246524,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246519,NaN,NaN,1.0
246521,NaN,NaN,24.0
246522,NaN,NaN,18.0
246523,NaN,NaN,1.0
246524,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246066,0.0,1.0,0.0
246067,0.0,1.0,0.0
246068,0.0,0.0,1.0
246278,0.0,1.0,0.0
246319,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246066,0,1,0
246067,0,1,0
246068,0,0,1
246278,0,1,0
246319,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246066,0,1,0
1,246067,0,1,0
2,246068,0,0,1
3,246278,0,1,0
4,246319,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246066,,1,
1,246067,,1,
2,246068,,,1
3,246278,,1,
4,246319,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 11:53AM,246524,18,Acute Outpatient Solutions
1,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix
2,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation
20,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc."
44,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc."
45,Sep 15 2022 11:24AM,246519,17,Beach Products Inc
46,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions
47,Sep 15 2022 11:18AM,246516,16,Sartorius Stedim North America
48,Sep 15 2022 11:11AM,246515,15,"Virtus Pharmaceuticals, LLC"
53,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 15 2022 11:53AM,246524,18,Acute Outpatient Solutions,,1,
1,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,,,1
2,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,,,18
3,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",,,24
4,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",,,1
5,Sep 15 2022 11:24AM,246519,17,Beach Products Inc,,,1
6,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions,,,1
7,Sep 15 2022 11:18AM,246516,16,Sartorius Stedim North America,,1,
8,Sep 15 2022 11:11AM,246515,15,"Virtus Pharmaceuticals, LLC",,,5
9,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:53AM,246524,18,Acute Outpatient Solutions,,1,
1,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1,,
2,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18,,
3,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24,,
4,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",1,,
5,Sep 15 2022 11:24AM,246519,17,Beach Products Inc,1,,
6,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions,1,,
7,Sep 15 2022 11:18AM,246516,16,Sartorius Stedim North America,,1,
8,Sep 15 2022 11:11AM,246515,15,"Virtus Pharmaceuticals, LLC",5,,
9,Sep 15 2022 11:04AM,246514,15,"Mizner Bioscience, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:53AM,246524,18,Acute Outpatient Solutions,,1,
1,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1,,
2,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18,,
3,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24,,
4,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:53AM,246524,18,Acute Outpatient Solutions,NaN,1.0,NaN
1,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18.0,NaN,NaN
3,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24.0,NaN,NaN
4,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:53AM,246524,18,Acute Outpatient Solutions,0.0,1.0,0.0
1,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18.0,0.0,0.0
3,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24.0,0.0,0.0
4,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4683383,111.0,17.0,7.0
15,739542,21.0,31.0,0.0
16,1478950,5.0,1.0,0.0
17,246519,1.0,0.0,0.0
18,246524,0.0,1.0,0.0
19,985979,2.0,2.0,0.0
20,246497,0.0,3.0,2.0
21,1969780,5.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4683383,111.0,17.0,7.0
1,15,739542,21.0,31.0,0.0
2,16,1478950,5.0,1.0,0.0
3,17,246519,1.0,0.0,0.0
4,18,246524,0.0,1.0,0.0
5,19,985979,2.0,2.0,0.0
6,20,246497,0.0,3.0,2.0
7,21,1969780,5.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,111.0,17.0,7.0
1,15,21.0,31.0,0.0
2,16,5.0,1.0,0.0
3,17,1.0,0.0,0.0
4,18,0.0,1.0,0.0
5,19,2.0,2.0,0.0
6,20,0.0,3.0,2.0
7,21,5.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,111.0
1,15,Released,21.0
2,16,Released,5.0
3,17,Released,1.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,18,19,20,21
Status,,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
Executing,17.0,31.0,1.0,0.0,1.0,2.0,3.0,3.0
Released,111.0,21.0,5.0,1.0,0.0,2.0,0.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,18,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,17.0,31.0,1.0,0.0,1.0,2.0,3.0,3.0
2,Released,111.0,21.0,5.0,1.0,0.0,2.0,0.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,18,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,17.0,31.0,1.0,0.0,1.0,2.0,3.0,3.0
2,Released,111.0,21.0,5.0,1.0,0.0,2.0,0.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()